<a href="https://colab.research.google.com/github/nataliyah123/ibia-Ersilia/blob/main/Automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic AutoML Example

In [ ]:
%%capture

!pip install rdkit-pypi

In [ ]:
%%capture

!pip install flaml

In [ ]:
import pandas as pd
from rdkit import Chem
from flaml import AutoML
from rdkit.Chem import rdMolDescriptors as rd
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
%%capture
!pip install git+https://github.com/samoturk/mol2vec

In [ ]:
smilescsv = pd.read_csv('dilismiles.csv', names = ['col_smiles'])
smileslist = smilescsv['col_smiles'].tolist()

In [ ]:
data = pd.read_csv('dili_padel_2d.csv')
dataframe = pd.DataFrame(data)
desc = pd.DataFrame(dataframe.iloc[:, 3:])

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (423,424,425,426,427,428,429,430,431,477,478,479,480,481,482,483,484,485,540,541,542,543,544,545,546,547,548,603,604,605,606,607,608,609,610,611,1012,1013,1050,1052,1074,1083,1084,1121,1123,1145,1217,1218,1219,1220,1222,1223,1224,1226,1227,1229) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
y = dataframe['Outcome']
y = np.array(y)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold

modelAutoML = AutoML(task="classification",time_budget=60)
cv = KFold(n_splits=10)
for i, (train, test) in enumerate(cv.split(desc, y)):
  modelAutoML.fit(desc.iloc[train], y[train], time_budget=20, verbose=-1)
  y_hat = modelAutoML.predict(desc.iloc[test])
  fpr, tpr, _ = roc_curve(y[test], y_hat)
  print("AUROC", auc(fpr, tpr))


AUROC 0.446256038647343
AUROC 0.5
AUROC 0.5057870370370371
AUROC 0.5
AUROC 0.5
AUROC 0.5432027649769585
AUROC 0.5246212121212122
AUROC 0.5177631578947368
AUROC 0.5277777777777778
AUROC 0.5398110661268556


# With Word2Vec feature selection method

In [ ]:
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence,DfVec, sentences2vec
import numpy as np
from gensim.models import word2vec

In [ ]:
model = word2vec.Word2Vec.load('model_300dim_eric.pkl')
smilescsv['mol'] = smilescsv['col_smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
smilescsv['sentence'] = smilescsv.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
smilescsv['mol2vec'] = [DfVec(x) for x in sentences2vec(smilescsv['sentence'], model, unseen='UNK')]

In [ ]:
X = np.array([x.vec for x in smilescsv['mol2vec']])

In [ ]:
X.shape

(588, 300)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold

modelAutoML = AutoML(task="classification",time_budget=60)
cv = KFold(n_splits=5)
for i, (train, test) in enumerate(cv.split(X, y)):
  modelAutoML.fit(X[train], y[train],verbose=-1)
  y_hat = modelAutoML.predict(X[test])
  fpr, tpr, _ = roc_curve(y[test], y_hat)
  print("AUROC", auc(fpr, tpr))


INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


AUROC 0.5


INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


AUROC 0.5625625625625625


INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


AUROC 0.5
AUROC 0.5045977011494253
AUROC 0.5445945945945946


More Information about the above warning can be found [here](https://github.com/microsoft/FLAML/wiki/About-%60low_cost_partial_config%60)